# Simulations for calibration (fringes on the FP)

#### Edited by Louise 16-04-2021
selfcal_lib.py allows to do several simulations concerning the calibration, especially the simlation of the signal on the focal plane. This notebook gives some examples of what you can do with.

To simulate the signal on the FP, there are 3 possibilities:
 - Analytical simulation (only fringes, with a simple sine) 
 - Qubic soft simulation
 - Maynooth simulations (plotted through Qubic soft)
 
Each possibility corresponds to a python class in Selfcal_lib.py.

In [ ]:
from __future__ import division, print_function

%matplotlib inline
%matplotlib notebook

import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from matplotlib.colors import SymLogNorm

from qubicpack.utilities import Qubic_DataDir
import qubic
from qubic import selfcal_lib as scal

plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:
# Get a dictionary
d = qubic.qubicdict.qubicDict()
d.read_from_file('global_source_oneDet.dict')
print(d['detarray'])
print(d['MultiBand'])
print(d['nf_sub'])

d['MultiBand'] = False
d['nf_sub'] = 1
d['config'] = 'FI'
d['beam_shape'] = 'gaussian'

## Instrument detector and horn config

In [ ]:
# QubicInstrument
q = qubic.QubicInstrument(d)

plt.figure(figsize=(6, 4))
q.detector.plot(fill=True)
plt.axis('square')
plt.xlabel('X_GRF')
plt.ylabel('Y_GRF')
plt.title('q.detector.plot()')

# q.optics.focal_length = 0.30 # Change the focal length

In [ ]:
# scal.close_switches(q, np.arange(1, 64))
# scal.open_switches(q, [1, 45, 50])
d1 = qubic.qubicdict.qubicDict()
d1.read_from_file('global_source_oneDet.dict')

d1['config'] = 'TD'
q1 = qubic.QubicInstrument(d1)

fig = plt.figure(figsize=(10, 10))
q.horn.plot(facecolor_open='grey', alpha=0.4)
q1.horn.plot(facecolor_open='g', alpha=0.3)

# plt.xlabel('$X_{GRF}$ [m]', fontsize=20)
# plt.ylabel('$Y_{GRF}$ [m]', fontsize=20)
plt.xlabel('$h_x$ [m]', fontsize=30)
plt.ylabel('$h_y$ [m]', fontsize=30)
plt.axis('square')
plt.rc('font', size=10) 
fig.tight_layout()
# plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/horn_array.pdf', dpi=100)

In [ ]:
# Open or close switches
# scal.close_switches(q, np.arange(1, 64))
# scal.open_switches(q, [1, 45, 50])

# Plot the horn array and a baseline
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
ax1, ax2 = np.ravel(axs)
scal.plot_horns(q, simple=False, ax=ax1)
# q.horn.plot()
ax1.set_xlabel('$X_{GRF}$ [m]', fontsize=20)
ax1.set_ylabel('$Y_{GRF}$ [m]', fontsize=20)


# Numbering on the horns
a = 1
x = q.horn.center[:, 0]
y = q.horn.center[:, 1]
# for i,j in zip(x, y):
#     corr = -0.005 # add a little correction to put annotation in marker's centrum
#     ax1.annotate(str(a),  xy=(i + corr, j + corr), fontsize=14, color='r')
#     a += 1

q.horn.plot(facecolor_open='green', alpha=0.2)
# scal.plot_baseline(q, [25, 57], ax=ax2)
ax2.set_xlabel('$X_{GRF}$ [m]', fontsize=20)
ax2.set_ylabel('$Y_{GRF}$ [m]', fontsize=20)

for i,j in zip(x, y):
    corr = 0.001 # add a little correction to put annotation in marker's centrum
    ax2.annotate(str(a),  xy=(i - (corr+0.001), j - corr), fontsize=10, color='k')
    a += 1
ax2.axis('square')

fig.tight_layout()

## Analytical function to simulate fringes
It is simply a 2D sine on the focal plane to make fringes according to the focal length, the frequency, the position of the source. 

A gaussian apodization is done to represent the primary beam. The Gaussian is centered in (0, 0) when the source is on-axis (theta_source=0, phi_source=0.). It moves according to the position of the source. 

In [ ]:
baseline = [60, 63]
Model_Ana = scal.Model_Fringes_Ana(q, baseline, 
                                   theta_source=np.deg2rad(0.), 
                                   phi_source=np.deg2rad(0.), 
                                   nu_source=150e9, fwhm=20., amp=1., frame='ONAFP')
print(Model_Ana.focal)

xONAFP, yONAFP, fringes = Model_Ana.get_fringes(times_gaussian=True)

scal.scatter_plot_FP(q, xONAFP, yONAFP, fringes, frame='ONAFP', s=50)


# Qubic soft simulation (no aberrations)

This is a simulation that doesn't take into account optical aberrations (only geometrical optics). If the point source is on the optical axis, then you get a symmetrical image.

In general Qubic Soft uses only the GRF frame. However, selfcal_lib allows to use the ONAFP frame which is the one uned on the instrument.

#### Get the image on the focal plane for a given horn configuration


In [ ]:
scal.open_switches(q, np.arange(1, 401, 1))
# scal.close_switches(q, np.arange(1, 64))

xONAFP, yONAFP, power = scal.get_response_power(q, 0., 0., 150e9, 1.)
# xONAFP, yONAFP, _ = scal.get_response_power(q, 0., 0., 150e9, 1.)

fig = plt.figure(figsize=(10, 10))
ax = fig.gca()
scal.scatter_plot_FP(q, xONAFP, yONAFP, power, frame='ONAFP', title='', s=190, unit=None, cbar=None,
                      vmin=0, vmax=1.5e-5, fig=fig, ax=ax, fontsize=30, Norm=SymLogNorm(1e-6))
fig.tight_layout()
plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/detector_array.pdf', dpi=100)

#### Create a Fringes object

In [ ]:
baseline = [274, 281] # for the FI
# baseline = [1, 57]     # for the TD

Model_QS = scal.Model_Fringes_QubicSoft(q, baseline,
                                        theta_source=0., phi_source=0.,
                                        nu_source=150e9, spec_irrad_source=1.,
                                        frame='ONAFP', external_A=None, hwp_position=0)

In [ ]:
# Information are store in the object
print(Model_QS.frame)
print(Model_QS.nu_source)
print(Model_QS.baseline)

#### Get fringes 

You can choose ONAFP or GRF frame.
Each pointing is defined by a position of the source (theta, phi).

In [ ]:
xONAFP, yONAFP, fringes = Model_QS.get_fringes(doplot=True, verbose=True, norm=None, s=30)

print('fringes shape:', fringes.shape)

In [ ]:
# Information are store in the object
print(Model_QS.x)
print(Model_QS.fringes)

#### Plot the fringes

There are 2 functions, one with scatter plot and the other with pcolor.
It is convenient to be sure of the coordinates and the image orientation. 

In [ ]:
fig, axs = plt.subplots(1, 2)
ax1, ax2 = np.ravel(axs)
fig.suptitle(f'Baseline: {baseline}', fontsize=18)
fig.subplots_adjust(wspace=0.3)

scal.scatter_plot_FP(q, xONAFP, yONAFP, fringes, frame='ONAFP', fig=fig, ax=ax1, 
                     title='Scatter plot', norm=None, s=30, cbar=True, fontsize=14)


scal.pcolor_plot_FP(q, xONAFP, yONAFP, fringes, frame='ONAFP', fig=fig, ax=ax2, 
                    title='pcolor plot', norm=None)

In [ ]:
# Make a plot for paper 3 without any axis
fig = plt.figure(figsize=(8, 8))
ax = plt.gca()
ax.axis('off')
scal.scatter_plot_FP(q, xONAFP, yONAFP, fringes, frame='ONAFP', fig=fig, ax=ax,
                     title='', norm=None, s=100, cbar=False, fontsize=18)
# fig.savefig('/home/lmousset/QUBIC/Images/plots/fringes_QSsimu_BaselineTD57-64.png', transparent=True)

#### Get fringes from the combination 
This is what is actually done when we measure the fringes.

In [ ]:
# Get all combinations
x, y, S, Cminus_i, Sminus_ij, Cminus_j, Ci, Cj, Sij = Model_QS.get_all_combinations_power(doplot=True, 
                                                                                          verbose=True, 
                                                                                          norm=SymLogNorm(5e-8), 
                                                                                          s=150)

print('shape of each combination :', Cminus_i.shape)

#### Look at the cycle acquired during measurement

In [ ]:
cycle = np.array((S, Cminus_i, Sminus_ij, S, Cminus_j, S, S)) - S
cycle.shape

In [ ]:
# TEs ans ASIC numbers
tes = scal.get_all_tes_numbers(q)

In [ ]:
for i in range(100, 102):
    plt.figure()
    plt.step(np.arange(0, 7), cycle[:, i], where='post', color='r')
    
    plt.axvspan(0, 1, color='k', alpha=0.2, label='All open')
    plt.axvspan(3, 4, color='k', alpha=0.2)
    plt.axvspan(5, 6, color='k', alpha=0.2)
    plt.axvspan(1, 2, color='b', alpha=0.2, label='H1 close')
    plt.axvspan(2, 3, color='g', alpha=0.2, label='H1 H2 close')
    plt.axvspan(4, 5, color='c', alpha=0.2, label='H2 close')
    
    plt.xlim(0, 6)
    plt.xlabel('Step index')
    plt.ylabel('TOD')
    plt.title(f'TES {tes[i, 1]} - ASIC {tes[i, 0]}')
    plt.legend()

In [ ]:
# Make the combination to get the fringes

# Complete one: S_tot - Cminus_i - Cminus_j + Sminus_ij + Ci +Cj
x, y, fringes_true = Model_QS.get_fringes_from_combination(measured_comb=False,
                                                           doplot=True, verbose=False, s=150)

# Measured one: S_tot - Cminus_i - Cminus_j + Sminus_ij
x, y, fringes_meas = Model_QS.get_fringes_from_combination(measured_comb=True,
                                                           doplot=True, verbose=False, s=150)

In [ ]:
# Look at the difference between both combinations

fig, axs = plt.subplots(1, 2)
ax1, ax2 = axs.ravel()
fig.suptitle(f'Baseline: {baseline}', fontsize=18)
fig.subplots_adjust(wspace=0.3)

scal.scatter_plot_FP(q, x, y, fringes_true, frame='ONAFP', 
                     fig=fig, ax=ax1, title='Complete combination', norm=None, s=150)

scal.scatter_plot_FP(q, x, y, fringes_meas, frame='ONAFP', 
                     fig=fig, ax=ax2, title='Measured combination', norm=None, s=150)

# Power on the focal plane with Maynooth simulations

Those simulations include optical aberrations.
You need to download the files at : https://drive.google.com/open?id=19dPHw_CeuFZ068b-VRT7N-LWzOL1fmfG

They are only available for the TD instrument.

In [ ]:
# Path to the simulated files 
rep = Qubic_DataDir(datafile='detcentres.txt', datadir='/home/lmousset/QUBIC/')
print('rep:', rep)

#### Get the power on the FP for an arbitrary horn configuration

This simulations are done in the ONAFP frame. The resolution is high (241 x 241) and it assumes an area bigger than the focal plane.

In [ ]:
open_horns = np.arange(1, 64)
xONAFP, yONAFP, power = scal.get_power_Maynooth(rep, open_horns, 
                                                0., 150e9, 
                                                q.horn.center,  
                                                hwp_position=0,
                                                verbose=True)
print(xONAFP.shape)
print(power.shape)

scal.scatter_plot_FP(q, xONAFP, yONAFP, power, s=1, 
                   frame='ONAFP', 
                   title='Maynooth full resolution', 
                   norm=None)

In [ ]:
# Make a plot for paper 3 without any axis
fig = plt.figure(figsize=(8, 8))
ax = fig.gca()
ax.axis('off')
scal.scatter_plot_FP(q, xONAFP, yONAFP, power, s=100, 
                   frame='ONAFP', 
                   title='', 
                   norm=None,
                   fig=fig, ax=ax,
                   cbar=False)
# fig.savefig('/home/lmousset/QUBIC/Images/plots/fringes_Maynoothsimu_BaselineTD57-64.png', transparent=True)

The following is an other way to do the same.

Qubic soft has an option `external_A` in `get_response()`. So I tried to use this option by passing Maynooth results in `external_A` but it is not fully working for now. There is a small bug.

In [ ]:
scal.open_switches(q, open_horns)

external_A = scal.make_external_A(rep, open_horns)

xONAFP, yONAFP, power = scal.get_response_power(q,
                                              theta=0., phi=0.,
                                              nu=150e9, spectral_irradiance=1.,
                                              frame='ONAFP',
                                              external_A=external_A, hwp_position=0, 
                                              verbose=True)

scal.scatter_plot_FP(q, xONAFP, yONAFP, power, s=100, 
                       frame='ONAFP', 
                       title='Maynooth full resolution', 
                       norm=None)

#### Get the fringes at TES resolution
To decrease the resolution, you can either make a real interpolation (`interp=True`) and then integrate or you can just make the mean inside each TES (`interp=False`). The second way is much faster.

In [ ]:
d['config'] = 'TD'
q = qubic.QubicInstrument(d)
baseline = [1, 57]  # for the TD

Model_Maynooth = scal.Model_Fringes_Maynooth(q, baseline, rep, 
                                             theta_source=0., nu_source=150e9,
                                             frame='ONAFP', interp=False)

In [ ]:
# Information are store in the object
print(Model_Maynooth.frame)
print(Model_Maynooth.nu_source)
print(Model_Maynooth.baseline)

In [ ]:
xONAFP, yONAFP, fringes = Model_Maynooth.get_fringes(verbose=True)

scal.scatter_plot_FP(q, xONAFP, yONAFP, fringes, s=200, 
                       frame='ONAFP', 
                       title='Maynooth at TES resolution', 
                       norm=None)

#### Get the fringes at TES reso doing the combination

You can make the "true" combination or the measured one.

In [ ]:
xONAFP, yONAFP, fringes_comb = Model_Maynooth.get_fringes_from_combination(measured_comb=True, verbose=True)

scal.scatter_plot_FP(q, xONAFP, yONAFP, fringes_comb, s=200, 
                   frame='ONAFP', 
                   title='Maynooth TES resolution', 
                   norm=None)